In [1]:
import os
import importlib
import logging
import time
importlib.reload(logging)
import framework
importlib.reload(framework)
import bert_qa
importlib.reload(bert_qa)
import infer_bert_qa
importlib.reload(infer_bert_qa)
import bert_utils
importlib.reload(bert_utils)
import pandas as pd
from framework import DataCuration, FeatureEngineering
from retrieval import TaskRetrieval, FeatureEngineeringRetrieval, Retrieval

# Define some constants and configurations
logging.getLogger().setLevel(logging.INFO)
ACCESS_TOKEN = 'WUpGevbWC9lsnTW8quNUtmWRdAEM89'

Using TensorFlow backend.


Set up the task details. This notebook handles Document Retrieval for CARTA dataset.


In [2]:
DATASET = 'carta' # supports w2 and resume
TASK_CONFIG = {
    'task': 'retrieval'
}

task = TaskRetrieval(TASK_CONFIG)

Set paths for datasets and goldens (local or ib, both work).
Specify configurations

In [3]:
CARTA_DATA = [
    # should contain only a single document, since the huge doc will be split into many segments (each treated as a new doc for retrieval)
   '/Users/ahsaasbajaj/Documents/Data/QA_model/data'
]

DATASET_CONFIG = {
    'path': CARTA_DATA,
    'is_local': True, 
    'file_type': 'ibocr',
    'identifier': lambda path: os.path.basename(path).split('.ibocr')[0],
    'convert2txt': True
}

CARTA_GOLDEN = None
GOLDEN_CONFIG = None

data = DataCuration(ACCESS_TOKEN, DATASET_CONFIG, GOLDEN_CONFIG)

INFO:root:Loading dataset from /Users/ahsaasbajaj/Documents/Data/QA_model/data
INFO:root:1 files loaded
INFO:root:Processing 1 IBOCR files to txt


In [4]:
data.dataset

{'annotated_AOI_4.pdf': <instabase.ocr.client.libs.ibocr.ParsedIBOCR at 0x154905828>}

In [5]:
filename = list(data.dataset.keys())[0]
print('filename to split and query: ', filename)

query = "Preferred Stocks"

NUM_FILES = len(data.dataset.keys())
stime = time.time()

DATA_ARGS = {
    'task': task,
    'dataset': data
}

TRAINING_ARGS = {
'model_file_or_path': "BM25Okapi"
}

fe = FeatureEngineeringRetrieval(DATA_ARGS)

corpus = fe.split_doc(filename=filename, split_size=100)  # list of document segments


filename to split and query:  annotated_AOI_4.pdf


In [6]:
tokenized_corpus = fe.tokenize_corpus(corpus)

model = Retrieval(DATA_ARGS, TRAINING_ARGS)
model.train(corpus, tokenized_corpus)
output = model.predict(query, len_results=5)

etime = time.time()
logging.info('Total time {} seconds'.format(etime - stime))

INFO:root:Total time 0.024983882904052734 seconds


In [7]:
output

[' Preferred Stock, Series A Preferred Stock  or Series A-1 Preferred Stock  (as applicable) that have',
 ' Seed Preferred Stock, Series A Preferred  Stock or Series A-1 Preferred Stock  (as applicable) into',
 ' Preferred Stock, $1.3049  per share forholders of Series A Preferred Stock and $2.3199 for',
 'series of Preferred Stock;',
 ' such Preferred Stock']